In [5]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.metrics  import silhouette_score
from scipy.spatial.distance import cdist
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [6]:
movie_for_ml = pd.read_csv('data/movies_for_ml.csv.zip')

In [7]:
movie_for_ml.head()

,tconst,originalTitle,startYear,nm0000005,nm0000008,nm0000019,nm0000033,nm0000036,nm0000037,nm0000040,...,Music,Musical,Mystery,News,Romance,Sci-Fi,Sport,Thriller,War,Western
0,tt0008879,Berg-Ejvind och hans hustru,1918,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,tt0009893,Die Austernprinzessin,1919,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,tt0009937,Blind Husbands,1919,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,tt0009968,Broken Blossoms or The Yellow Man and the Girl,1919,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,tt0010247,Herr Arnes pengar,1919,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
cols = movie_for_ml.columns[2:]
X = movie_for_ml[cols]
X


,startYear,nm0000005,nm0000008,nm0000019,nm0000033,nm0000036,nm0000037,nm0000040,nm0000041,nm0000045,...,Music,Musical,Mystery,News,Romance,Sci-Fi,Sport,Thriller,War,Western
0,1918,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1919,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1919,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,1919,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,1919,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11636,2019,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
11637,2019,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
11638,2019,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
11639,2020,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
